In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import random
import re
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

os.chdir('C:/Users/simjh96/Documents/Car_Accident/data/crawling/GIS')

In [ ]:
def _get_chromebrowser():
    print("시스템>>> 현재 디렉토리: {}".format(os.getcwd()))
    print("시스템>>> 디렉토리가 맞다면(Y) 아니면(N): ")
    inp = input()
    if inp == 'N':
        print("시스템>>> 크롬드라이버가 있는 위치 입력...")
        os.chdir(input())
    browser = webdriver.Chrome("./chromedriver.exe")
    #self 로 바꿔 arg 에도 넣고
    print("시스템>>> 크롬드라이버를 실행 합니다...")
    return browser

In [ ]:
#in page
browser = _get_chromebrowser()
browser.get('http://taas.koroad.or.kr/gis/mcm/mcl/initMap.do?menuId=GIS_GMP_STS_RSN')

In [ ]:
#경기도 오산시 어린이 사고 수동 설정 후 실행(수동 작업 필요)
browser.find_element_by_link_text('검색').click()
time.sleep(5)

In [ ]:
#totalcount
print('시스템>>> total count를 셉니다...')
soup = BeautifulSoup(browser.page_source, "lxml")
tot = soup.find(attrs='total-count')
tot = int(re.findall(r'[0-9]+',tot.text)[0])
print('시스템>>> 총 {}건을 검색합니다'.format(tot))

#저장될 cords
cords = []
    
#1계층
#페이지 넘어가게 loop로 묶어
for i in range(tot//10 + 1):
    print('시스템>>> {} 페이지로 넘어갑니다.'.format(str(i+1)))   
    browser.find_element_by_partial_link_text('{}'.format(str(i+1))).click()    
    
    #2계층
    # 10 건별 클릭
    elms = browser.find_elements_by_class_name('iStyle')
    for j in range(len(elms)):
        time.sleep(np.random.uniform(0.5,1))
        print('시스템>>> {}번째 건을 추출합니다...'.format(j+1))
        elms[j].click()
        WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'pop-headerN')))
        
        #BBOX 추출
        #수정필요>>>첫 BBOX 가 검색전 메인 화면 좌표가 나와서 좌표가 한칸씩 밀림
        print('시스템>>> BBOX 좌표를 구합니다...')
        soup = BeautifulSoup(browser.page_source, "lxml")
        imgs = soup.find_all(attrs='olTileImage')
        for img in imgs:
            if ('BBOX' in img['src']):
                full_cord = re.findall(r'BBOX=[0-9.,]*&',img['src'])[0][5:-1]
                cord_list = re.findall('[0-9]+.[0-9]+',full_cord)
                cords.append(((float(cord_list[0])+float(cord_list[2]))/2, 
                              (float(cord_list[1])+float(cord_list[3]))/2,
                              soup.find(attrs='td1').text))
                break

cords_df = pd.DataFrame(cords)
with open("{}.csv".format(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M')), "w", encoding="utf8") as f:
    f.write(cords_df.to_csv())

print('시스템>>> 모든 수집을 완료했습니다!')

In [ ]:
#merge all
cords_df = pd.read_csv('2017-2019.csv')
cords_df = cords_df.dropna()
cords_df.index = range(len(cords_df))
cords_df = cords_df.loc[:,('0','1')]
cords_df.columns = ['x','y']

b_df = pd.read_csv('2017-2019_b.csv')

df = pd.concat((b_df,cords_df),axis=1)
#=========================================
cords_df = pd.read_csv('2014-2016.csv')
cords_df = cords_df.dropna()
cords_df.index = range(len(cords_df))
cords_df = cords_df.loc[:,('0','1')]
cords_df.columns = ['x','y']

b_df = pd.read_csv('2014-2016_b.csv')

df = pd.concat((df,pd.concat((b_df,cords_df),axis=1)))

cords_df = pd.read_csv('2011-2013.csv')
cords_df = cords_df.dropna()
cords_df.index = range(len(cords_df))
cords_df = cords_df.loc[:,('0','1')]
cords_df.columns = ['x','y']

b_df = pd.read_csv('2011-2013_b.csv')

df = pd.concat((df,pd.concat((b_df,cords_df),axis=1)))

cords_df = pd.read_csv('2008-2010.csv')
cords_df = cords_df.dropna()
cords_df.index = range(len(cords_df))
cords_df = cords_df.loc[:,('0','1')]
cords_df.columns = ['x','y']

b_df = pd.read_csv('2008-2010_b.csv')

df = pd.concat((df,pd.concat((b_df,cords_df),axis=1)))

cords_df = pd.read_csv('2007.csv')
cords_df = cords_df.dropna()
cords_df.index = range(len(cords_df))
cords_df = cords_df.loc[:,('0','1')]
cords_df.columns = ['x','y']

b_df = pd.read_csv('2007_b.csv')

df = pd.concat((df,pd.concat((b_df,cords_df),axis=1)))

with open("{}_concated.csv".format(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M')), "w", encoding="utf8") as f:
    f.write(df.to_csv())

In [ ]:
#lat to lon
#대차대조
cords_df = pd.read_csv('2014-2016.csv')
cords_df = cords_df.dropna()
cords_df.index = range(len(cords_df))
cords_df = cords_df.loc[:,('0','1')]


lat1 = 37.18451855217116
lat2 = 37.181550525972085
lat3 = 37.13463907826839
lat4 = 37.1894037929
lat5 = 37.169842533

lon1 = 127.0093782249407
lon2 = 127.0305097105699
lon3 = 127.08264073187888
lon4 = 127.05884081658
lon5 = 127.05268917

utm1_x = 323301.52
utm2_x = 325170.67
utm3_x = 329693.51
utm4_x = 327703.69
utm5_x = 327112.99

utm1_y = 4117197.57
utm2_y = 4116829.06
utm3_y = 4111529.12
utm4_y = 4117648.49
utm5_y = 4115489.32


x1 = cords_df.loc[24][0]
x2 = cords_df.loc[0][0]
x3 = cords_df.loc[133][0]
x4 = cords_df.loc[121][0]
x5 = cords_df.loc[107][0]

y1 = cords_df.loc[24][1]
y2 = cords_df.loc[0][1]
y3 = cords_df.loc[133][1]
y4 = cords_df.loc[121][1]
y5 = cords_df.loc[107][1]

# (utm2_x-utm1_x) = b1*(x2-x1) + c1*(y2-y1)
# (utm3_x-utm1_x)  = b1*(x3-x1) + c1*(y3-y1)
b1 = ((utm2_x-utm1_x)/(y2-y1) - (utm3_x-utm1_x)/(y3-y1))/((x2-x1)/(y2-y1) - (x3-x1)/(y3-y1))
c1 = ((utm2_x-utm1_x) - b1*(x2-x1))/(y2-y1)

b2 = ((utm2_y-utm1_y)/(y2-y1) - (utm3_y-utm1_y)/(y3-y1))/((x2-x1)/(y2-y1) - (x3-x1)/(y3-y1))
c2 = ((utm2_y-utm1_y) - b2*(x2-x1))/(y2-y1)

# b1*(x5-x1) + c1*(y5-y1) + utm1_x
# b2*(x5-x1) + c2*(y5-y1) + utm1_y
# utm5_x
# utm5_y

In [ ]:
#14-16에서 모수를 전체 07-19에 적용해도 되겠지? 시스템이 같으니까?
cords_df = pd.read_csv('2021_01_10_18_41_concated.csv').iloc[:,-2:]

cords_df['e'] = cords_df.apply(lambda xy: b1*(xy[0]-x1) + c1*(xy[1]-y1) + utm1_x, axis=1)
cords_df['s'] = cords_df.apply(lambda xy: b2*(xy[0]-x1) + c2*(xy[1]-y1) + utm1_y, axis=1)

In [ ]:
from pyproj import Proj
myProj = Proj("+proj=utm +zone=52N, +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
lon, lat = myProj(cords_df['e'].values, cords_df['s'].values, inverse=True)

df = pd.DataFrame([lat,lon]).T
df.columns = ['lat','lon']

final_df = pd.concat([pd.read_csv('2021_01_10_18_41_concated.csv').iloc[:,1:-2],df],axis=1)

with open("{}_final.csv".format(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M')), "w", encoding="utf8") as f:
    f.write(final_df.to_csv())